Copyright **`(c)`** 2021 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see 'LICENCE.md' for details.

# Connect 4

In [176]:
from collections import Counter
import numpy as np
from IPython.display import clear_output

In [177]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"

## Basic Functions

In [178]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

## Montecarlo Evaluation

In [179]:
def _mc(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
            return p
    return 0


def montecarlo(board, player):
    montecarlo_samples = 100
    cnt = Counter(_mc(np.copy(board), player) for _ in range(montecarlo_samples))
    return (cnt[1] - cnt[-1]) / montecarlo_samples


def eval_board(board, player):
    if four_in_a_row(board, 1):
        # Alice won
        return 1
    elif four_in_a_row(board, -1):
        # Bob won
        return -1
    else:
        # Not terminal, let's simulate...
        return montecarlo(board, player)

## MinMax

In [180]:
MAX_DEPTH = 2

def eval_board_minmax(board):
    if four_in_a_row(board, 1):
        # Alice won
        return 1
    elif four_in_a_row(board, -1):
        # Bob won
        return -1
    else:
        return 0

def is_leaf_node(board, player):
    return four_in_a_row(board, player) or not valid_moves(board)

def minmax(board, player, alpha, beta, depth):

    possible = valid_moves(board)
    leaf = is_leaf_node(board, player)

    if depth == 0 or leaf:
        if leaf:
            if four_in_a_row(board, player):
                return None, 1000000000000000
            elif four_in_a_row(board, -player):
                return None, -1000000000000000
            else:
                return None, 0
        else:
            return None, eval_board(board, player)

    if possible:
        np.random.shuffle(possible) # Add some randomness
    
    if player == 1: # 1 is always maximizing player
        value = np.NINF
        column = np.random.choice(possible)
        for col in possible:
            play(board, col, player)
            _, score = minmax(board, -player, alpha, beta, depth-1)
            # print(score)
            take_back(board, col)
            if score > value:
                value = score
                column = col
            alpha = max(value, alpha)

            if alpha >= beta:
                break
        return column, value


    else:   # minimizing player
        value = np.Inf
        column = np.random.choice(possible)
        for col in possible:
            play(board, col, player)
            _, score = minmax(board, -player, alpha, beta, depth-1)
            # print(score)
            take_back(board, col)
            if score < value:
                value = score
                column = col
            alpha = min(value, alpha)

            if alpha >= beta:
                break
        return column, value
             

## Example

In [181]:
board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
print(board)

# Insert here your moves for starting with a predefined configuration.
# AI is always player 1

player = 1
while (not four_in_a_row(board, 1) or not four_in_a_row(board, -1)) and valid_moves(board):
    best_move, eval = minmax(board, player, np.NINF, np.Inf, MAX_DEPTH)
    print(best_move, eval)
    if best_move == None:
        break
    play(board, best_move, player)

    clear_output(wait=True) # Just for output clearness

    print(board)

    if not four_in_a_row(board, 1) and not four_in_a_row(board, -1) and valid_moves(board):
        try:
            human = int(input("\nTake your time and tell me the column if which you want to play: "))
            play(board, human, -player)
        except KeyboardInterrupt:
            pass
    else:
        break
    
    print(board)

print(f'\n{board}')
won = eval_board_minmax(board)
if won:
    print(f'Player {won} won! Congratulations!')
else:
    print('Draw.')



[[-1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]

[[-1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]


1